In [1]:
# Import necessary libraries
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchinfo import summary
from itertools import product
import os

from helper_functions import set_seeds

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Set random seeds for reproducibility
def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

# Load pretrained ViT model
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# Freeze base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# Define dataset paths
train_dir = r'C:\Users\satya\Desktop\ransom\cnn2\MORE\RGBprocessed_images\train'
test_dir = r'C:\Users\satya\Desktop\ransom\cnn2\MORE\RGBprocessed_images\test'
val_dir = r'C:\Users\satya\Desktop\ransom\cnn2\MORE\RGBprocessed_images\val'



# Get automatic transforms from pretrained ViT weights
pretrained_vit_transforms = pretrained_vit_weights.transforms()

# Function to create dataloaders
def create_dataloaders(train_dir, val_dir, test_dir, transform, batch_size, num_workers=os.cpu_count()):
    # Create datasets
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    val_data = datasets.ImageFolder(val_dir, transform=transform)  # Validation dataset
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Create dataloaders
    train_dataloader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True
    )
    val_dataloader = DataLoader(
        val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
    )
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
    )

    return train_dataloader, val_dataloader, test_dataloader, class_names

# Define hyperparameter grid
hyperparam_grid = {
    'learning_rate': [1e-3, 1e-4],  # Learning rates to test
    'batch_size': [32, 64],         # Batch sizes to test
    'optimizer': ['Adam', 'SGD'],   # Optimizers to test
    'weight_decay': [0, 1e-4]       # Weight decay (regularization)
}

# Generate all combinations of hyperparameters
grid_combinations = [dict(zip(hyperparam_grid.keys(), values)) 
                     for values in product(*hyperparam_grid.values())]

# Track best model and metrics
best_metrics = {
    'accuracy': 0,
    'hyperparams': None,
    'model_state': None
}

# Grid search loop
for params in grid_combinations:
    print(f"\n\033[1mTesting hyperparameters: {params}\033[0m")
    
    # Set seeds for reproducibility
    set_seeds()
    
    # Create dataloaders with current batch size
    train_dataloader, val_dataloader, test_dataloader, class_names = create_dataloaders(
        train_dir=train_dir,
        val_dir=val_dir,
        test_dir=test_dir,
        transform=pretrained_vit_transforms,
        batch_size=params['batch_size']
    )
    
    # Initialize fresh model for each combination
    model = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)
    for parameter in model.parameters():
        parameter.requires_grad = False
    model.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)
    
    # Configure optimizer
    optimizer_config = {
        'params': model.parameters(),
        'lr': params['learning_rate'],
        'weight_decay': params['weight_decay']
    }
    if params['optimizer'] == 'Adam':
        optimizer = torch.optim.Adam(**optimizer_config)
    elif params['optimizer'] == 'SGD':
        optimizer = torch.optim.SGD(**optimizer_config, momentum=0.9)
    
    # Train model with validation
    from going_modular.going_modular import engine
    print(f"Training with {params['optimizer']} optimizer...")
    results = engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=val_dataloader,  # Validate on validation set
        optimizer=optimizer,
        loss_fn=nn.CrossEntropyLoss(),
        epochs=10,
        device=device
    )
    
    # Track best model using validation accuracy
    current_val_acc = max(results['test_acc'])
    if current_val_acc > best_metrics['accuracy']:
        best_metrics['accuracy'] = current_val_acc
        best_metrics['hyperparams'] = params
        best_metrics['model_state'] = model.state_dict()
        print(f"New best validation accuracy: {current_val_acc:.4f}")

# Final evaluation with best model
print("\n\033[1mBest hyperparameters:", best_metrics['hyperparams'])
print(f"Best validation accuracy: {best_metrics['accuracy']:.4f}\033[0m")

# Load best model for test evaluation
best_model = torchvision.models.vit_b_16().to(device)
best_model.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)
best_model.load_state_dict(best_metrics['model_state'])

# Evaluate on test set
test_results = engine.train(
    model=best_model,
    train_dataloader=train_dataloader,  # Not used but required by function
    test_dataloader=test_dataloader,
    optimizer=optimizer,  # Not used in evaluation
    loss_fn=nn.CrossEntropyLoss(),
    epochs=1,  # Only evaluation
    device=device
)
print(f"\n\033[1mFinal test accuracy: {max(test_results['test_acc']):.4f}\033[0m")

Using device: cuda

Testing hyperparameters: {'learning_rate': 0.001, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 0}
Training with Adam optimizer...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.2649 | train_acc: 0.9102 | test_loss: 0.1271 | test_acc: 0.9630
Epoch: 2 | train_loss: 0.1277 | train_acc: 0.9631 | test_loss: 0.0878 | test_acc: 0.9803
Epoch: 3 | train_loss: 0.0957 | train_acc: 0.9760 | test_loss: 0.0708 | test_acc: 0.9873
Epoch: 4 | train_loss: 0.0793 | train_acc: 0.9791 | test_loss: 0.0556 | test_acc: 0.9942
Epoch: 5 | train_loss: 0.0705 | train_acc: 0.9817 | test_loss: 0.0485 | test_acc: 0.9954
Epoch: 6 | train_loss: 0.0622 | train_acc: 0.9844 | test_loss: 0.0393 | test_acc: 0.9942
Epoch: 7 | train_loss: 0.0567 | train_acc: 0.9864 | test_loss: 0.0382 | test_acc: 0.9954
Epoch: 8 | train_loss: 0.0539 | train_acc: 0.9864 | test_loss: 0.0332 | test_acc: 0.9954
Epoch: 9 | train_loss: 0.0525 | train_acc: 0.9867 | test_loss: 0.0338 | test_acc: 0.9954
Epoch: 10 | train_loss: 0.0464 | train_acc: 0.9887 | test_loss: 0.0347 | test_acc: 0.9942
New best validation accuracy: 0.9954

Testing hyperparameters: {'learning_rate': 0.001, 'batch_size': 32, 'op

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.2649 | train_acc: 0.9102 | test_loss: 0.1271 | test_acc: 0.9630
Epoch: 2 | train_loss: 0.1277 | train_acc: 0.9631 | test_loss: 0.0878 | test_acc: 0.9803
Epoch: 3 | train_loss: 0.0957 | train_acc: 0.9760 | test_loss: 0.0708 | test_acc: 0.9873
Epoch: 4 | train_loss: 0.0793 | train_acc: 0.9791 | test_loss: 0.0557 | test_acc: 0.9942
Epoch: 5 | train_loss: 0.0706 | train_acc: 0.9817 | test_loss: 0.0486 | test_acc: 0.9954
Epoch: 6 | train_loss: 0.0623 | train_acc: 0.9844 | test_loss: 0.0393 | test_acc: 0.9942
Epoch: 7 | train_loss: 0.0568 | train_acc: 0.9864 | test_loss: 0.0383 | test_acc: 0.9954
Epoch: 8 | train_loss: 0.0539 | train_acc: 0.9864 | test_loss: 0.0333 | test_acc: 0.9954
Epoch: 9 | train_loss: 0.0526 | train_acc: 0.9867 | test_loss: 0.0340 | test_acc: 0.9954
Epoch: 10 | train_loss: 0.0465 | train_acc: 0.9887 | test_loss: 0.0348 | test_acc: 0.9942

Testing hyperparameters: {'learning_rate': 0.001, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0}
T

...

Epoch: 1 | train_loss: 0.7081 | train_acc: 0.4707 | test_loss: 0.6266 | test_acc: 0.6004
Epoch: 2 | train_loss: 0.5784 | train_acc: 0.8524 | test_loss: 0.5167 | test_acc: 0.8996
Epoch: 3 | train_loss: 0.4920 | train_acc: 0.8930 | test_loss: 0.4436 | test_acc: 0.9029
Epoch: 4 | train_loss: 0.4340 | train_acc: 0.9022 | test_loss: 0.3935 | test_acc: 0.9241
Epoch: 5 | train_loss: 0.3928 | train_acc: 0.9095 | test_loss: 0.3567 | test_acc: 0.9252
Epoch: 6 | train_loss: 0.3619 | train_acc: 0.9104 | test_loss: 0.3279 | test_acc: 0.9275
Epoch: 7 | train_loss: 0.3369 | train_acc: 0.9133 | test_loss: 0.3061 | test_acc: 0.9286
Epoch: 8 | train_loss: 0.3188 | train_acc: 0.9116 | test_loss: 0.2874 | test_acc: 0.9286
Epoch: 9 | train_loss: 0.3020 | train_acc: 0.9142 | test_loss: 0.2732 | test_acc: 0.9286
Epoch: 10 | train_loss: 0.2888 | train_acc: 0.9173 | test_loss: 0.2598 | test_acc: 0.9342

Testing hyperparameters: {'learning_rate': 0.0001, 'batch_size': 64, 'optimizer': 'SGD', 'weight_decay': 0.0

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.7081 | train_acc: 0.4707 | test_loss: 0.6266 | test_acc: 0.6004
Epoch: 2 | train_loss: 0.5784 | train_acc: 0.8524 | test_loss: 0.5167 | test_acc: 0.8996
Epoch: 3 | train_loss: 0.4920 | train_acc: 0.8930 | test_loss: 0.4436 | test_acc: 0.9029
Epoch: 4 | train_loss: 0.4340 | train_acc: 0.9022 | test_loss: 0.3935 | test_acc: 0.9241
Epoch: 5 | train_loss: 0.3928 | train_acc: 0.9095 | test_loss: 0.3567 | test_acc: 0.9252
Epoch: 6 | train_loss: 0.3619 | train_acc: 0.9104 | test_loss: 0.3279 | test_acc: 0.9275
Epoch: 7 | train_loss: 0.3369 | train_acc: 0.9133 | test_loss: 0.3061 | test_acc: 0.9286
Epoch: 8 | train_loss: 0.3188 | train_acc: 0.9116 | test_loss: 0.2874 | test_acc: 0.9286
Epoch: 9 | train_loss: 0.3021 | train_acc: 0.9142 | test_loss: 0.2732 | test_acc: 0.9286
Epoch: 10 | train_loss: 0.2888 | train_acc: 0.9173 | test_loss: 0.2598 | test_acc: 0.9342

Best hyperparameters: {'learning_rate': 0.001, 'batch_size': 64, 'optimizer': 'Adam', 'weight_decay': 0}
Bes

  0%|          | 0/1 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 4.30 GiB is allocated by PyTorch, and 178.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:

# Evaluate on test set
test_results = engine.train(
    model=best_model,
    train_dataloader=train_dataloader,  # Not used but required by function
    test_dataloader=test_dataloader,
    optimizer=optimizer,  # Not used in evaluation
    loss_fn=nn.CrossEntropyLoss(),
    epochs=1,  # Only evaluation
    device=device
)
print(f"\n\033[1mFinal test accuracy: {max(test_results['test_acc']):.4f}\033[0m")

  0%|          | 0/1 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 4.33 GiB is allocated by PyTorch, and 141.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)